In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
import torch

In [ ]:
# Sample data
sample_data = [
    {"customer": "I'm having trouble logging into my account.", "agent_response": "Sure, could you please provide your email address?"},
    {"customer": "My order hasn't arrived yet.", "agent_response": "Can you please provide your order number so I can check the status?"},
    {"customer": "I need to update my billing information.", "agent_response": "Sure, could you please provide the new billing address?"},
    # Add more sample conversations here
]
 

In [ ]:
# Prepare input-output pairs
train_data = [{"input_text": conv["customer"], "target_text": conv["agent_response"]} for conv in sample_data]
 

In [ ]:
# Load pre-trained T5 model and tokenizer
model_name = "t5-small"  # or "t5-base", "t5-large", etc.
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
 

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_dir='./logs',
    logging_steps=100,
    save_steps=1000,
    evaluation_strategy="steps",
    eval_steps=500,
    output_dir='./models'
)

In [ ]:
 
# Define training data
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
 
    def __len__(self):
        return len(self.data)
 
    def __getitem__(self, idx):
        item = self.data[idx]
        input_text = item['input_text']
        target_text = item['target_text']
 
        input_encoding = self.tokenizer(input_text, truncation=True, max_length=self.max_length, padding="max_length", return_tensors="pt")
        target_encoding = self.tokenizer(target_text, truncation=True, max_length=self.max_length, padding="max_length", return_tensors="pt")
 
        labels = target_encoding["input_ids"]
        labels[labels == 0] = -100  # Ignore padding tokens in loss calculation
 
        return {
            "input_ids": input_encoding["input_ids"].flatten(),
            "attention_mask": input_encoding["attention_mask"].flatten(),
            "labels": labels.flatten(),
        }

In [ ]:
train_dataset = CustomDataset(train_data, tokenizer)

In [ ]:
# Define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
)
 

In [ ]:
# Fine-tune the model
trainer.train()

In [ ]:
# Save the fine-tuned model
trainer.save_model("./fine_tuned_model")

In [ ]:
# Load the fine-tuned model
fine_tuned_model = T5ForConditionalGeneration.from_pretrained("./fine_tuned_model")
 
# Function to generate response
def generate_response(input_text, model, tokenizer, max_length=50):
    input_encoding = tokenizer(input_text, truncation=True, max_length=max_length, padding="max_length", return_tensors="pt")
    input_ids = input_encoding.input_ids.to(model.device)
    attention_mask = input_encoding.attention_mask.to(model.device)
    output_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=max_length, num_beams=5, early_stopping=True)
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return output_text
 
# Test the model
test_input = "I want to track my order."
generated_response = generate_response(test_input, fine_tuned_model, tokenizer)
print("Generated response:", generated_response)